In [1]:
interp.repositories() ++= Seq(coursierapi.MavenRepository.of(
"https://jitpack.io"
))

In [2]:
import $ivy. `org.carbonateresearch::conus:0.2.2`

import $ivy.$                                    

In [3]:
import org.carbonateresearch.conus._
import org.carbonateresearch.conus.common.ModelResults
import org.carbonateresearch.conus.modelzoo.GeneralGeology._
import org.carbonateresearch.conus.modelzoo.PasseyHenkesClumpedDiffusionModel._
import org.carbonateresearch.conus.simulators.AkkaCentralSimulatorActor
import org.carbonateresearch.conus.{AllCells, ModelVariable, PerCell, Step, SteppedModel}

import math._

import org.carbonateresearch.conus._

import org.carbonateresearch.conus.common.ModelResults

import org.carbonateresearch.conus.modelzoo.GeneralGeology._

import org.carbonateresearch.conus.modelzoo.PasseyHenkesClumpedDiffusionModel._

import org.carbonateresearch.conus.simulators.AkkaCentralSimulatorActor

import org.carbonateresearch.conus.{AllCells, ModelVariable, PerCell, Step, SteppedModel}


import math._

In [6]:
 val sim = new AlmondSimulator

// a few constants
val rhocal:Double = 2.71 //Density of carbonates
val cOf=889000 //concentration of O in fluid
val cCf=200 //concentration of C in fluid
val cOcal = 480000 //concentration of O in stoichiometric calcite
val cCcal = 120000 //concentration of C in stoichiometric calcite
val d13Ctdc = 0 //carbon isotope composition of total dissolved carbon in fluid

// setting model variables
val initialAge:ModelVariable[Double] = ModelVariable("Initial age",96.0,"Ma")
val finalAge:ModelVariable[Double] = ModelVariable("Final age",0.0,"Ma")
val d18Of:ModelVariable[Double] = ModelVariable("Fluid d18O",-1.0,"‰")
val TmaxSample:ModelVariable[Double] = ModelVariable("Tmax",0.0,"˚C")
val WFF:ModelVariable[Double] = ModelVariable("Weight fraction of fluid",0.0,"")
val CWRR:ModelVariable[Double] = ModelVariable("Cumulative water/rock ratio",0.0,"")
val FluidMass:ModelVariable[Double] = ModelVariable("Mass of fluid",0.2,"")
val d18Occ:ModelVariable[Double] = ModelVariable("d18Occ",1.0,"‰")
val d13Ccc:ModelVariable[Double] = ModelVariable("d13Ccc",1.0,"‰")
val D47r:ModelVariable[Double] = ModelVariable("D47 with partial recrystallization",.5,"‰")
val fractionRec:ModelVariable[Double] = ModelVariable("Fraction of recrystallization",0.01,"‰")
val initalBurialAtModelStart:ModelVariable[Double] = ModelVariable("Initial burial at model start",0.0,"meters")

// Initialise model conditions as lists
val burialHistory = List((96.0,0.0), (39.0,2000.0), (0.0,-70.0))
val geothermalGradientHistory= List((96.0,30.0),(0.0,30.0))
val surfaceTemperaturesHistory = List((96.0,30.0),(0.0,30.0))
val numberOfSteps = 10
val ageList:List[Double] = List(97.0,96.0,95.0)
val finalAgeList:List[Double] = List(0.0)
val rangeOfFluidMasses:List[Double] = List(0.2)
val rangeOfPartialRecrystallization = List(1.0,.5,.1,.01,0.0)

// Bulk isotope calculation equations
val d18OccFunction = (s:Step) => {
val cO=(WFF(s)*cOf)+(1-WFF(s))*cOcal
val alpha = math.exp((18.03*(1000*pow((burialTemperature(s)+273.15), -1)) - 32.42)/1000)
val d18O = ((d18Of(s) * WFF(s) * cOf) + d18Occ(s) * (1-WFF(s)) * cOf)/ cO
((d18O * cOf * alpha) - (1000 * cOf * WFF(s) * (1-alpha))) / ((cOcal * (1-WFF(s)) * alpha) + cOf * WFF(s))
}

val d13CccFunction = (s:Step) => {
val cC=(WFF(s)*cCf)+(1-WFF(s))*cCcal
val alpha = exp((-2.4612+(7666.3/100) - (2.9880*pow(10,3)/pow((burialTemperature(s)+273.15),6)))/1000)
val d13C = ((d13Ctdc * WFF(s) * d13Ctdc) + (d13Ccc(s) * (1-WFF(s)) * cCcal))/cC
((d13C * cC * alpha) - (1000 * cCf * WFF(s) * (1-alpha))) / (cCcal * (1-WFF(s) * alpha) + cCf * WFF(s))
}

val eaglefordModel = new SteppedModel(numberOfSteps,"Eagleford recrystallization")
.setGrid(19)
.defineMathematicalModel(
  age =>> ageOfStep(initialAge,finalAge),
  depth =>> {(s:Step) => {burialDepthFromAgeModel(age,burialHistory).apply(s)+initalBurialAtModelStart(s)}},
  surfaceTemperature =>> surfaceTemperaturesAtAge(age, surfaceTemperaturesHistory),
  geothermalGradient =>> geothermalGradientAtAge(age,geothermalGradientHistory),
  burialTemperature =>> burialTemperatureFromGeothermalGradient(surfaceTemperature,depth,geothermalGradient),
  dT =>> dTFun,
  D47eq =>> D47eqFun,
  D47i =>> D47iFun,
  SampleTemp =>> davies19_T,
  FluidMass =>> {(s:Step) => 0.2},
  d18Of =>> {(s:Step) => -1.0},
  WFF =>> {(s:Step) => FluidMass(s) / (FluidMass(s) + rhocal)},
  CWRR =>> {(s:Step) => s.stepNumber * WFF(s)/(1-WFF(s))},
  d18Occ =>> d18OccFunction,
  d13Ccc =>> d13CccFunction,
  D47r =>> {(s:Step) => {if(burialTemperature(s)-burialTemperature(s-1) >= 0){D47r(s-1) * (1.0-fractionRec(s)) + D47eq(s) * fractionRec(s)}
  else {D47r(s-1)}}}
)
.defineInitialModelConditions(
  AllCells(initialAge,ageList),
  AllCells(finalAge,finalAgeList),
  AllCells(fractionRec,rangeOfPartialRecrystallization),
  AllCells(D47i,List(0.670,0.680,0.690)),
  AllCells(D47r,List(0.670,0.680,0.690)),
  PerCell(initalBurialAtModelStart,List(
    (List(0.0),Seq(0)),
    (List(-0.91),Seq(1)),
    (List(-1.22),Seq(2)),
    (List(-2.90),Seq(3)),
    (List(-4.57),Seq(4)),
    (List(-4.88),Seq(5)),
    (List(-4.90),Seq(6)),
    (List(-6.10),Seq(7)),
    (List(-6.40),Seq(8)),
    (List(-6.71),Seq(9)),
    (List(-25.90),Seq(10)),
    (List(-28.96),Seq(11)),
    (List(-29.87),Seq(12)),
    (List(-35.05),Seq(13)),
    (List(-35.36),Seq(14)),
    (List(-35.97),Seq(15)),
    (List(-36.58),Seq(16)),
    (List(-37.19),Seq(17)),
    (List(-38.90),Seq(18)))))
.defineCalibration(
  D47r.isBetween(0.511,0.683).atCells(Seq(0)),
  D47r.isBetween(0.511,0.683).atCells(Seq(1)),
  D47r.isBetween(0.511,0.683).atCells(Seq(2)),
  D47r.isBetween(0.511,0.683).atCells(Seq(3)),
  D47r.isBetween(0.511,0.683).atCells(Seq(4)),
  D47r.isBetween(0.511,0.683).atCells(Seq(5)),
  D47r.isBetween(0.511,0.683).atCells(Seq(6)),
  D47r.isBetween(0.511,0.683).atCells(Seq(7)),
  D47r.isBetween(0.511,0.683).atCells(Seq(8)),
  D47r.isBetween(0.511,0.683).atCells(Seq(9)),
  D47r.isBetween(0.511,0.683).atCells(Seq(10)),
  D47r.isBetween(0.511,0.683).atCells(Seq(11)),
  D47r.isBetween(0.511,0.683).atCells(Seq(12)),
  D47r.isBetween(0.511,0.683).atCells(Seq(13)),
  D47r.isBetween(0.511,0.683).atCells(Seq(14)),
  D47r.isBetween(0.511,0.683).atCells(Seq(15)),
  D47r.isBetween(0.511,0.683).atCells(Seq(16)),
  D47r.isBetween(0.511,0.683).atCells(Seq(17)),
  D47r.isBetween(0.511,0.683).atCells(Seq(18))
)

sim.evaluate(eaglefordModel)

A total of 135 unique models were defined, attempting to create a list now.
Models list successfully created.


Run progress: #################################################################################################### 100.0% Total runtime: 0 seconds

Variable Initial age with initial value of 96.0 Ma defined

Variable Final age with initial value of 0.0 Ma defined

Variable Fluid d18O with initial value of -1.0 ‰ defined

...

Variable D47 with partial recrystallization with initial value of 0.5 ‰ defined

Variable Fraction of recrystallization with initial value of 0.01 ‰ defined

Variable Initial burial at model start with initial value of 0.0 meters defined

Feature,Value
Name,Eagleford recrystallization
Nb of steps,10
Nb of models,135
Nb grid cells,19
Nb of operations per step,16
Total nb of operations,410400


sim: AlmondSimulator = org.carbonateresearch.conus.AlmondSimulator@4845abc6
rhocal: Double = 2.71
cOf: Int = 889000
cCf: Int = 200
cOcal: Int = 480000
cCcal: Int = 120000
d13Ctdc: Int = 0
burialHistory: List[(Double, Double)] = List(
  (96.0, 0.0),
  (39.0, 2000.0),
  (0.0, -70.0)
)
geothermalGradientHistory: List[(Double, Double)] = List(
  (96.0, 30.0),
  (0.0, 30.0)
)
surfaceTemperaturesHistory: List[(Double, Double)] = List(
  (96.0, 30.0),
  (0.0, 30.0)
)
numberOfSteps: Int = 10
ageList: List[Double] = List(97.0, 96.0, 95.0)
finalAgeList: List[Double] = List(0.0)
rangeOfFluidMasses: List[Double] = List(0.2)
rangeOfPartialRecrystallization: List[Double] = List(1.0, 0.5, 0.1, 0.01, 0.0)
d18OccFunction: Step => Double = ammonite.$sess.cmd5$Helper$$Lambda$3019/0x000000080183f840@21645260
d13CccFunction: Step => Double = ammonite.$sess.cmd5$Helper$$Lambda$3020/0x000000080183e840@64bc7da9

In [7]:
Thread.sleep(2000)
val results: ModelResults = sim.getResults(eaglefordModel)
results.getModel(0)

Eagleford recrystallization Contains 135 models each comprising 10 timesteps Model# Initial model conditions RSME 7 Initial age 97.0 at all cells Final age 0.0 at all cells Fraction of recrystallization 0.1 at all cells Δ47i 0.67 at all cells D47 with partial recrystallization 0.67 at all cells Initial burial at model start 0.0 at cell (0), -0.91 at cell (1), -1.22 at cell (2), -2.9 at cell (3), -4.57 at cell (4), -4.88 at cell (5), -4.9 at cell (6), -6.1 at cell (7), -6.4 at cell (8), -6.71 at cell (9), -25.9 at cell (10), -28.96 at cell (11), -29.87 at cell (12), -35.05 at cell (13), -35.36 at cell (14), -35.97 at cell (15), -36.58 at cell (16), -37.19 at cell (17), -38.9 at cell (18), 0.0 6 Initial age 95.0 at all cells Final age 0.0 at all cells Fraction of recrystallization 0.5 at all cells Δ47i 0.67 at all cells D47 with partial recrystallization 0.67 at all cells Initial burial at model start 0.0 at cell (0), -0.91 at cell (1), -1.22 at cell (2), -2.9 at cell (3), -4.57 at cell (4), -4.88 at cell (5), -4.9 at cell (6), -6.1 at cell (7), -6.4 at cell (8), -6.71 at cell (9), -25.9 at cell (10), -28.96 at cell (11), -29.87 at cell (12), -35.05 at cell (13), -35.36 at cell (14), -35.97 at cell (15), -36.58 at cell (16), -37.19 at cell (17), -38.9 at cell (18), 0.0 3 Initial age 95.0 at all cells Final age 0.0 at all cells Fraction of recrystallization 1.0 at all cells Δ47i 0.67 at all cells D47 with partial recrystallization 0.67 at all cells Initial burial at model start 0.0 at cell (0), -0.91 at cell (1), -1.22 at cell (2), -2.9 at cell (3), -4.57 at cell (4), -4.88 at cell (5), -4.9 at cell (6), -6.1 at cell (7), -6.4 at cell (8), -6.71 at cell (9), -25.9 at cell (10), -28.96 at cell (11), -29.87 at cell (12), -35.05 at cell (13), -35.36 at cell (14), -35.97 at cell (15), -36.58 at cell (16), -37.19 at cell (17), -38.9 at cell (18), 0.0 1 Initial age 97.0 at all cells Final age 0.0 at all cells Fraction of recrystallization 1.0 at all cells Δ47i 0.67 at all cells D47 with partial recrystallization 0.67 at all cells Initial burial at model start 0.0 at cell (0), -0.91 at cell (1), -1.22 at cell (2), -2.9 at cell (3), -4.57 at cell (4), -4.88 at cell (5), -4.9 at cell (6), -6.1 at cell (7), -6.4 at cell (8), -6.71 at cell (9), -25.9 at cell (10), -28.96 at cell (11), -29.87 at cell (12), -35.05 at cell (13), -35.36 at cell (14), -35.97 at cell (15), -36.58 at cell (16), -37.19 at cell (17), -38.9 at cell (18), 0.0 8 Initial age 96.0 at all cells Final age 0.0 at all cells Fraction of recrystallization 0.1 at all cells Δ47i 0.67 at all cells D47 with partial recrystallization 0.67 at all cells Initial burial at model start 0.0 at cell (0), -0.91 at cell (1), -1.22 at cell (2), -2.9 at cell (3), -4.57 at cell (4), -4.88 at cell (5), -4.9 at cell (6), -6.1 at cell (7), -6.4 at cell (8), -6.71 at cell (9), -25.9 at cell (10), -28.96 at cell (11), -29.87 at cell (12), -35.05 at cell (13), -35.36 at cell (14), -35.97 at cell (15), -36.58 at cell (16), -37.19 at cell (17), -38.9 at cell (18), 0.0 5 Initial age 96.0 at all cells Final age 0.0 at all cells Fraction of recrystallization 0.5 at all cells Δ47i 0.67 at all cells D47 with partial recrystallization 0.67 at all cells Initial burial at model start 0.0 at cell (0), -0.91 at cell (1), -1.22 at cell (2), -2.9 at cell (3), -4.57 at cell (4), -4.88 at cell (5), -4.9 at cell (6), -6.1 at cell (7), -6.4 at cell (8), -6.71 at cell (9), -25.9 at cell (10), -28.96 at cell (11), -29.87 at cell (12), -35.05 at cell (13), -35.36 at cell (14), -35.97 at cell (15), -36.58 at cell (16), -37.19 at cell (17), -38.9 at cell (18), 0.0 4 Initial age 97.0 at all cells Final age 0.0 at all cells Fraction of recrystallization 0.5 at all cells Δ47i 0.67 at all cells D47 with partial recrystallization 0.67 at all cells Initial burial at model start 0.0 at cell (0), -0.91 at cell (1), -1.22 at cell (2), -2.9 at cell (3), -4.57 at cell (4), -4.88 at cell (5), -4.9 at cell (6), -6.1 at cell (7), -6.4 

Cell coordinates,Initial age,Final age,Fraction of recrystallization,Δ47i,D47 with partial recrystallization,Initial burial at model start,Age,Burial depth,Surface Temperature,Geothermal gradient,Burial Temperature,dT,Δ47eq,Sample temperature,Mass of fluid,Fluid d18O,Weight fraction of fluid,Cumulative water/rock ratio,d18Occ,d13Ccc
(0),97.0,0.0,0.1,0.6699983124826556,0.6384546821304563,0.0,0.0,-70.0,30.0,30.0,27.9,5.412062769230768E14,0.6821990379119576,32.119279348078464,0.2,-1.0,0.06872852233676977,0.6642066420664207,5.918001697432124,1.0925744726145907
(1),97.0,0.0,0.1,0.6699983124826134,0.6384593931260506,0.0,0.0,-70.0,30.0,30.0,27.9,5.412062769230768E14,0.6821990379119576,32.11927934809336,0.2,-1.0,0.06872852233676977,0.6642066420664207,5.918001697432124,1.0925744726145907
(2),97.0,0.0,0.1,0.6699983124825992,0.6384609982623941,0.0,0.0,-70.0,30.0,30.0,27.9,5.412062769230768E14,0.6821990379119576,32.11927934809842,0.2,-1.0,0.06872852233676977,0.6642066420664207,5.918001697432124,1.0925744726145907
(3),97.0,0.0,0.1,0.6699983124825215,0.6384696996452683,0.0,0.0,-70.0,30.0,30.0,27.9,5.412062769230768E14,0.6821990379119576,32.11927934812587,0.2,-1.0,0.06872852233676977,0.6642066420664207,5.918001697432124,1.0925744726145907
(4),97.0,0.0,0.1,0.6699983124824442,0.638478353552758,0.0,0.0,-70.0,30.0,30.0,27.9,5.412062769230768E14,0.6821990379119576,32.11927934815316,0.2,-1.0,0.06872852233676977,0.6642066420664207,5.918001697432124,1.0925744726145907
(5),97.0,0.0,0.1,0.66999831248243,0.6384799604407634,0.0,0.0,-70.0,30.0,30.0,27.9,5.412062769230768E14,0.6821990379119576,32.11927934815816,0.2,-1.0,0.06872852233676977,0.6642066420664207,5.918001697432124,1.0925744726145907
(6),97.0,0.0,0.1,0.6699983124824291,0.638480064116056,0.0,0.0,-70.0,30.0,30.0,27.9,5.412062769230768E14,0.6821990379119576,32.1192793481585,0.2,-1.0,0.06872852233676977,0.6642066420664207,5.918001697432124,1.0925744726145907
(7),97.0,0.0,0.1,0.6699983124823736,0.6384862857647852,0.0,0.0,-70.0,30.0,30.0,27.9,5.412062769230768E14,0.6821990379119576,32.119279348178054,0.2,-1.0,0.06872852233676977,0.6642066420664207,5.918001697432124,1.0925744726145907
(8),97.0,0.0,0.1,0.6699983124823596,0.638487841524737,0.0,0.0,-70.0,30.0,30.0,27.9,5.412062769230768E14,0.6821990379119576,32.119279348183,0.2,-1.0,0.06872852233676977,0.6642066420664207,5.918001697432124,1.0925744726145907
(9),97.0,0.0,0.1,0.6699983124823454,0.6384894492895292,0.0,0.0,-70.0,30.0,30.0,27.9,5.412062769230768E14,0.6821990379119576,32.11927934818806,0.2,-1.0,0.06872852233676977,0.6642066420664207,5.918001697432124,1.0925744726145907
